In [ ]:
%pip install ffmpeg-python


  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [1]:
import ffmpeg
import os

# Extract Photos From Videos

In [ ]:
os.makedirs("Img", exist_ok=True)

In [ ]:
def extract_images_from_video(video_folder, video_name, output_folder_name, fps =1, crop_width=953, crop_height=700, output_width=1024, output_height=1024):
    # Create output directory
    output_path = os.path.join('Img', output_folder_name)
    os.makedirs(output_path, exist_ok=True)

    # Define crop filter with scaling
    # First crop, then scale
    filter_chain = f"crop={crop_width}:{crop_height}:647:100,scale={output_width}:{output_height}"

    # Input video and output frames with cropping and scaling
    video = ffmpeg.input(os.path.join(os.getcwd(), video_folder, video_name))
    video.output(
        os.path.join(output_path, 'frame_%04d.png'),
        vf=f"fps={fps},{filter_chain}"
    ).run()

    print(f"Frames cropped, scaled, and saved to {output_path}")

In [ ]:
# Extracting the photos form the videos
extract_images_from_video('wire_video_parts', 'wire vidoe 1 12_53_09 - 12_53_18.mp4', 'wire_images_video_1')
extract_images_from_video('wire_video_parts', 'wire vidoe 2 12_53_24 - 12_53_52.mp4', 'wire_images_video_2')
extract_images_from_video('wire_video_parts', 'wire vidoe 3 12_53_52 - 12_54_04.mp4', 'wire_images_video_3')
extract_images_from_video('wire_video_parts', 'wire vidoe 4 12_55_06 - 12_55_13.mp4', 'wire_images_video_4')
extract_images_from_video('wire_video_parts', 'wire vidoe 5 12_55_24 - 12_55_35.mp4', 'wire_images_video_5')
extract_images_from_video('wire_video_parts', 'wire vidoe 6 12_55_38 - 12_55_42.mp4', 'wire_images_video_6')
extract_images_from_video('wire_video_parts', 'wire vidoe 7 12_55_49 - 12_55_51.mp4', 'wire_images_video_7')
extract_images_from_video('wire_video_parts', 'wire vidoe 8 12_56_14 - 12_56_54.mp4', 'wire_images_video_8')

d:\Python projects\arteries_image_segmentation\input.mp4


FileNotFoundError: [WinError 2] The system cannot find the file specified

# Creating Ground Truth Binary Masks For The Images

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import json
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
def plot_pair(images, gray=False):

    fig, axes = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(10,8))
    i=0
    
    for y in range(2):
        if gray:
            axes[y].imshow(images[i], cmap='gray')
        else:
            axes[y].imshow(images[i])
        axes[y].get_xaxis().set_visible(False)
        axes[y].get_yaxis().set_visible(False)
        i+=1
    
    plt.show()
    

def get_poly(ann_path):
    
    with open(ann_path) as handle:
        data = json.load(handle)
    
    shape_dicts = data['shapes']
    
    return shape_dicts


def create_binary_masks(im, shape_dicts):

    # Creates a black image of size im
    blank = np.zeros(shape=(im.shape[0], im.shape[1]), dtype=np.float32)
    
    # Paints the wanted shape in white by the polygon points
    for shape in shape_dicts:
        points = np.array(shape['points'], dtype=np.int32)
        cv2.fillPoly(blank, [points], 255)
        
    return blank

In [3]:
annotation_dirs = list(filter(lambda dname: dname.endswith('annotated'),os.listdir('Img')))
annotation_dirs = sorted(annotation_dirs, key=lambda dname: int(dname.split('_')[3]))
annotation_dict = {dname: os.listdir(f'Img/{dname}') for dname in annotation_dirs}
annotation_dict.keys()

dict_keys(['wire_images_video_1_annotated', 'wire_images_video_2_annotated', 'wire_images_video_3_annotated', 'wire_images_video_4_annotated', 'wire_images_video_5_annotated', 'wire_images_video_6_annotated', 'wire_images_video_7_annotated', 'wire_images_video_8_annotated'])

In [4]:
image_dirs = list(filter(lambda dname: dname[-1].isdigit(), os.listdir('Img')))
image_dirs = sorted(image_dirs, key=lambda dname: int(dname[-1]))
images_dict = {dname: os.listdir(f'Img/{dname}') for dname in image_dirs}

In [11]:
total_images = 0
for img_list in images_dict.values():
    total_images += len(img_list)
print(f"Total images: {total_images}") 

Total images: 89


In [12]:

total_binary_masks = 0
# For each image directory and the corresponding annotation directory
for image_dir, annotation_dir in zip(images_dict, annotation_dict):
    # Create a new directory for storing the ground truth binary masks of the corresponding image directory
    new_directory_path = os.path.join('Img', f'{image_dir}_gtmasks')
    os.makedirs(new_directory_path, exist_ok=True)
    # For each image file and the corresponding annotation file
    for image_name, annotation_name in zip(images_dict[image_dir], annotation_dict[annotation_dir]):
        # Load the image file
        im = cv2.imread(os.path.join("Img", image_dir, image_name), 0)
        ann_path = os.path.join('Img', annotation_dir, annotation_name)
        # Load the polygons from the annotation file
        shape_dicts = get_poly(ann_path)
        # Create a binary mask for the image by the polygons
        im_binary = create_binary_masks(im, shape_dicts)
        """plot_pair([im, im_binary], gray=True)
        plt.show()"""
        im_binary_name = image_name.split('.')[0] + "_gtmask.png"
        save_path = os.path.join(new_directory_path, im_binary_name)
        # Save the binary mask in the corresponding new directory
        cv2.imwrite(save_path, im_binary)
        print(f"Created image at: {save_path}")
        total_binary_masks += 1
print(f"Total binary masks: {total_binary_masks}")
        

Created image at: Img\wire_images_video_1_gtmasks\wire1_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_1_gtmasks\wire2_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_1_gtmasks\wire3_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_1_gtmasks\wire4_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_1_gtmasks\wire5_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_1_gtmasks\wire6_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_2_gtmasks\wire10_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_2_gtmasks\wire11_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_2_gtmasks\wire12_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_2_gtmasks\wire13_ultrasound_watertank_gtmask.png
Created image at: Img\wire_images_video_2_gtmasks\wire14_ultrasound_watertank_gtmask.png
Created image at: Img\wire_

# Next Header